In [1]:
import pandas as pd

In [2]:
folder = "Result_SWGS28"
file_prefix = "SWZcurdata_"



In [3]:
def load_and_clean(file):
    df = pd.read_csv(file)
    cur_data_raw = df["curData"].tolist()
    #print (cur_data_raw)
    clean_data = []
    for x in cur_data_raw:
        if "b'\\x" in x:
            #print ("hex")
            clean_data.append(x[4:6])
        else :
            
            clean_data.append(str(hex(ord(x[2])))[2:])
    #print (clean_data)
    df["cleanData"] = clean_data
    return df

In [7]:
a_file = folder+"/"+file_prefix+str(0)

In [8]:
all_data = []
all_df = pd.DataFrame()

In [9]:
for i in range(32):
    file = folder+"/"+file_prefix+str(i)
    df = load_and_clean(file)
    all_df[str(i)] = df["cleanData"]
    all_data.append(df)
    


In [10]:
one_df = load_and_clean(a_file)

In [11]:
all_df["Timestamp"] = one_df["Timestamp"]

In [12]:
all_df["date-time"] = one_df["date-time"]

In [13]:
beacons = []
for index, row in all_df.iterrows():
    x = ""
    for i in range(32):
        x = x+row[str(i)]
    beacons.append(x)
len(beacons)
all_df["beacons"] = beacons

In [15]:
all_df.to_csv("Swiss_beacons_combined.csv")


## Beacon Matchin code here

In [47]:
#nus_df = pd.read_csv("NUS_beacons_combined.csv")
swiss_df = pd.read_csv("Swiss_beacons_combined.csv")

/home/odin/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4,6,14,15,20,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
nus_df = pd.read_csv("NUS_beacons_combined.csv")

In [49]:
nus_beacons = nus_df["beacons"].unique().tolist()

In [50]:
swiss_beacons = swiss_df["beacons"].unique().tolist()

In [51]:
same_beacons = []
for x in nus_beacons:
    for y in swiss_beacons:
        if x == y : 
            print ("match",x)
            same_beacons.append(x)

match ff00000000000000000000000000000000000000000000000000000000000000
match 0000000000000000000000000000000000000000000000000000000000000000
match 8e9befc918d6d62327465b05e194130807a3f9d615708e3d9720f450d2a37f3e
match df9a2a92e84213907e06ae4b7b15cc9ee44dac9390ec0ed94bcd045f7a265cbd
match ccdbd21ba31ae07bc84267a7e0a5e2f40f2bcf92b94a1d1ef6c885fa2a070035
match 587431e5f03e489198aa27f53fc7525c62ec9bd6126d94d60fd21f31e09f52ae
match d423d4c3538c2d4c6cbe3f0e145ddd8f59035dc9004177ed6ad86326a916b289
match c6b6a31df3acbafba30f35a1f675d595042cc527ca64f125b4654165a4754e91
match abb9b5484e86b5fcbce9e16c4e9178f7d82fc4750f3b5c3ca5e6ce5eb687222d


In [24]:
Singapore_date_time = []
Singapoer_tiemstamp = []
for bx in same_beacons:
    
    for index, row in nus_df.iterrows():
        if row["beacons"] == bx :
            Singapore_date_time.append(row["date-time"])
            Singapoer_tiemstamp.append(row["Timestamp"])
            break


In [30]:
Swiss_date_time = []
Swiss_tiemstamp = []
for bx in same_beacons:
    
    for index, row in swiss_df.iterrows():
        if row["beacons"] == bx :
            Swiss_date_time.append(row["date-time"])
            Swiss_tiemstamp.append(row["Timestamp"])
            break


In [32]:
Swiss_date_time

['2020-02-25 15:01:31',
 '2020-02-27 14:51:01',
 '2020-02-21 20:12:30',
 '2020-02-18 22:48:00',
 '2020-02-14 23:14:00',
 '2020-02-12 23:25:00',
 '2020-02-12 00:13:00',
 '2020-02-10 23:36:00',
 '2019-10-02 05:57:00']

In [52]:
def find_matched_time(same_beacons,df):
    date_time_list = []
    tiemstamps_list = []
    for bx in same_beacons:
        for index, row in df.iterrows():
            if row["beacons"] == bx :
                date_time_list.append(row["date-time"])
                tiemstamps_list.append(row["Timestamp"])
                break
    return date_time_list, tiemstamps_list
            

In [53]:
Singapore_date_time, Singapor_tiemstamp = find_matched_time(same_beacons,nus_df)

In [54]:
Swiss_date_time, Swiss_tiemstamp = find_matched_time(same_beacons,swiss_df)

In [55]:
final_df = pd.DataFrame()

In [58]:
final_df["Singapore_date_time"]  = Singapore_date_time
final_df["Singapor_tiemstamp"] = Singapor_tiemstamp
final_df["Swiss_date_time"]  = Swiss_date_time
final_df["Swiss_tiemstamp"] = Swiss_tiemstamp
final_df["beacons"] = same_beacons


In [59]:
final_df.to_csv("SG_SWISS_matchedBeacons.csv")